In [1]:
!which pip3

/home/mgross/anaconda3/envs/contrast-net/bin/pip3


In [2]:
! nvidia-smi

Thu Jan 29 05:26:35 2026       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro RTX 8000                Off | 00000000:17:00.0 Off |                  Off |
| 33%   29C    P8              23W / 260W |   3136MiB / 49152MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import IPython
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import os
import pandas as pd
import pytorch_lightning
from monai.visualize import img2tensorboard
import torch
import time
import sklearn

from torch.utils.data import DataLoader

import monai
from monai.transforms import \
    Compose, LoadImaged, AddChanneld, Orientationd, \
    NormalizeIntensityd, ScaleIntensityRangePercentilesd, \
    SpatialPadd, Spacingd, Resized, \
    RandCropByPosNegLabeld, ToTensord,  \
    DataStatsd, AsDiscrete, KeepLargestConnectedComponent, RandFlipd
#LoadNiftid


from monai.data import list_data_collate
from monai.networks.layers import Norm

# from monai.metrics import compute_meandice
from monai.transforms import KeepLargestConnectedComponent

from monai.metrics import DiceMetric
from monai.inferers import sliding_window_inference
from monai.utils import set_determinism
from monai.metrics import compute_meandice
from monai.metrics import compute_hausdorff_distance
from monai.metrics import compute_average_surface_distance


from monai.data import write_nifti
from tqdm.notebook import tqdm





monai.config.print_config()

MONAI version: 0.7.0
Numpy version: 1.20.3
Pytorch version: 1.8.2
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: bfa054b9c3064628a21f4c35bbe3132964e91f43

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 3.2.1
scikit-image version: 0.18.3
Pillow version: 8.3.1
Tensorboard version: 2.6.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.9.2
tqdm version: 4.62.3
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: NOT INSTALLED or UNKNOWN VERSION.
pandas version: 1.3.4
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [4]:
print('CUDA available: ', torch.cuda.is_available())

n_gpus = torch.cuda.device_count()
for i in range(n_gpus):
    print('GPU %d: %s' % (i, torch.cuda.get_device_name(i)))


CUDA available:  True
GPU 0: Quadro RTX 8000


## Load the Data

In [5]:
INPUT_PATH = '../ContrastNets.xlsx'

df_gt = pd.read_excel(INPUT_PATH, index_col=False)
df_gt.rename(columns={'PV' : 'IMAGE', 'SEGM': 'GT-SEGM'}, inplace = True)

In [6]:
df_pre = pd.read_csv('../Results/Results-PRE-Net.csv')
df_art = pd.read_csv('../Results/Results-ART-Net.csv')
df_pv =  pd.read_csv('../Results/Results-PV-Net.csv')
df_del = pd.read_csv('../Results/Results-DEL-Net.csv')

In [7]:
for i in [df_pre]:
    for idx, row in  i.iterrows():
            i.loc[idx, 'ACCESSION_ID'] = str(row['AUTOMATED-LIVER-SEGM'].split('/')[-1].split('_')[-5])


In [8]:
for i in [df_art]:
    for idx, row in  i.iterrows():
            i.loc[idx, 'ACCESSION_ID'] = str(row['AUTOMATED-LIVER-SEGM'].split('/')[-1].split('_')[-4])#.split('_')[-5])


In [9]:
for i in [df_pv, df_del]:
    for idx, row in  i.iterrows():
            i.loc[idx, 'ACCESSION_ID'] = str(row['AUTOMATED-LIVER-SEGM'].split('/')[-1].split('_')[-5])


In [10]:
df_pre.rename(columns={'AUTOMATED-LIVER-SEGM' : 'PRE-SEGM'}, inplace = True)
df_art.rename(columns={'AUTOMATED-LIVER-SEGM' : 'ART-SEGM'}, inplace = True)
df_pv.rename(columns={'AUTOMATED-LIVER-SEGM' : 'PV-SEGM'}, inplace = True)
df_del.rename(columns={'AUTOMATED-LIVER-SEGM' : 'DEL-SEGM'}, inplace = True)


In [11]:
df = pd.merge(df_pre, df_art, on = 'ACCESSION_ID')
df = pd.merge(df, df_pv, on = 'ACCESSION_ID')
df = pd.merge(df, df_del, on = 'ACCESSION_ID')
df = pd.merge(df, df_gt, on = 'ACCESSION_ID')

/tmp/ipykernel_9088/947928985.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'HD95_x', 'POST_DICE_x', 'InferenceTime_x', 'EXPERIMENT_ID_x', 'DICE_x', 'Unnamed: 0_x', 'POST_HD95_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, df_del, on = 'ACCESSION_ID')


In [12]:
fusion_data_dict = df.to_dict('records')

/tmp/ipykernel_9088/680658821.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  fusion_data_dict = df.to_dict('records')


## Setup Input Transformations

In [14]:
SPACING = (2.0,2.0,2.0)
fusion_keys = ['GT-SEGM','PRE-SEGM','ART-SEGM', 'PV-SEGM', 'DEL-SEGM']


late_fusion_transforms = Compose([
    LoadImaged(keys=fusion_keys, reader='NibabelReader', as_closest_canonical=True),
    AddChanneld(keys=fusion_keys),
    Orientationd(keys=fusion_keys, axcodes='RAS'),
    Spacingd(keys=fusion_keys, pixdim=SPACING, mode='nearest', padding_mode='reflection'),
    ])



## Create a Persistent Data Cache for Training and Validation Data

In [15]:
def compute_evaluation_metrics(y_pred, y, keep=False):
    
    argmax = monai.transforms.AsDiscrete(argmax=True)
    one_hot = monai.transforms.AsDiscrete(to_onehot=True, n_classes=2)
    dice = monai.metrics.DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
    hd = monai.metrics.HausdorffDistanceMetric(include_background=False, reduction="mean", percentile=95, get_not_nans=False)
    mad = monai.metrics.SurfaceDistanceMetric(include_background=False, reduction="mean", get_not_nans=False)
    keep_largest = monai.transforms.KeepLargestConnectedComponent(applied_labels=[1])

    dice_list = list()
    hd_list = list()
    mad_list = list()
    for y_pred_i, y_i in monai.data.decollate_batch((y_pred, y)):
        #print(y_pred_i.shape)
        #print(y_i.shape)

        y_pred_i_proc = monai.transforms.AddChannel()(one_hot(y_pred_i))
        #print(y_pred_i_proc.shape)
        
        y_i_proc = monai.transforms.AddChannel()(one_hot(y_i))
        #print(y_i_proc.shape)
            
        #dice(y_pred=y_pred_i_proc, y=one_hot(y_i))
        #hd(y_pred=y_pred_i_proc, y=one_hot(y_i))
        #mad(y_pred=y_pred_i_proc, y=one_hot(y_i))
            
        dice(y_pred=y_pred_i_proc, y=y_i_proc)
        hd(y_pred=y_pred_i_proc, y=y_i_proc)
        mad(y_pred=y_pred_i_proc, y=y_i_proc)
        
        dice_value = dice.aggregate().item()
        hd_value = hd.aggregate().item()
        mad_value = mad.aggregate().item()
        
        #print(dice_value)
        #print(hd_value)
        #print(mad_value)
        
        dice_list.append(dice_value)
        hd_list.append(hd_value)
        mad_list.append(mad_value)
              
    return dice_list, hd_list, mad_list


In [16]:
fusion_ds = monai.data.Dataset(
    data=fusion_data_dict, transform=late_fusion_transforms)

fusion_loader = DataLoader(fusion_ds, batch_size=1, shuffle=False, num_workers=4, collate_fn=list_data_collate)

In [17]:
#dsc
dice_segm_2mpMRI_pre_art_values = list()
dice_segm_2mpMRI_pre_pv_values = list()
dice_segm_2mpMRI_pre_del_values = list()
dice_segm_2mpMRI_art_pv_values = list()
dice_segm_2mpMRI_art_del_values = list()
dice_segm_2mpMRI_pv_del_values = list()


dice_segm_3mpMRI_pre_art_pv_values = list()
dice_segm_3mpMRI_pre_art_del_values = list()
dice_segm_3mpMRI_pre_pv_del_values = list()
dice_segm_3mpMRI_art_pv_del_values = list()

dice_segm_4mpMRI_values = list()

#hd
hd_segm_2mpMRI_pre_art_values = list()
hd_segm_2mpMRI_pre_pv_values = list()
hd_segm_2mpMRI_pre_del_values = list()
hd_segm_2mpMRI_art_pv_values = list()
hd_segm_2mpMRI_art_del_values = list()
hd_segm_2mpMRI_pv_del_values = list()


hd_segm_3mpMRI_pre_art_pv_values = list()
hd_segm_3mpMRI_pre_art_del_values = list()
hd_segm_3mpMRI_pre_pv_del_values = list()
hd_segm_3mpMRI_art_pv_del_values = list()

hd_segm_4mpMRI_values = list()

#
mad_segm_2mpMRI_pre_art_values = list()
mad_segm_2mpMRI_pre_pv_values = list()
mad_segm_2mpMRI_pre_del_values = list()
mad_segm_2mpMRI_art_pv_values = list()
mad_segm_2mpMRI_art_del_values = list()
mad_segm_2mpMRI_pv_del_values = list()


mad_segm_3mpMRI_pre_art_pv_values = list()
mad_segm_3mpMRI_pre_art_del_values = list()
mad_segm_3mpMRI_pre_pv_del_values = list()
mad_segm_3mpMRI_art_pv_del_values = list()

mad_segm_4mpMRI_values = list()

In [18]:
acc_ids = list()


#dsc
dice_segm_2mpMRI_pre_art_values = list()
dice_segm_2mpMRI_pre_pv_values = list()
dice_segm_2mpMRI_pre_del_values = list()
dice_segm_2mpMRI_art_pv_values = list()
dice_segm_2mpMRI_art_del_values = list()
dice_segm_2mpMRI_pv_del_values = list()


dice_segm_3mpMRI_pre_art_pv_values = list()
dice_segm_3mpMRI_pre_art_del_values = list()
dice_segm_3mpMRI_pre_pv_del_values = list()
dice_segm_3mpMRI_art_pv_del_values = list()

dice_segm_4mpMRI_values = list()

#hd
hd_segm_2mpMRI_pre_art_values = list()
hd_segm_2mpMRI_pre_pv_values = list()
hd_segm_2mpMRI_pre_del_values = list()
hd_segm_2mpMRI_art_pv_values = list()
hd_segm_2mpMRI_art_del_values = list()
hd_segm_2mpMRI_pv_del_values = list()


hd_segm_3mpMRI_pre_art_pv_values = list()
hd_segm_3mpMRI_pre_art_del_values = list()
hd_segm_3mpMRI_pre_pv_del_values = list()
hd_segm_3mpMRI_art_pv_del_values = list()

hd_segm_4mpMRI_values = list()

#mad
mad_segm_2mpMRI_pre_art_values = list()
mad_segm_2mpMRI_pre_pv_values = list()
mad_segm_2mpMRI_pre_del_values = list()
mad_segm_2mpMRI_art_pv_values = list()
mad_segm_2mpMRI_art_del_values = list()
mad_segm_2mpMRI_pv_del_values = list()


mad_segm_3mpMRI_pre_art_pv_values = list()
mad_segm_3mpMRI_pre_art_del_values = list()
mad_segm_3mpMRI_pre_pv_del_values = list()
mad_segm_3mpMRI_art_pv_del_values = list()

mad_segm_4mpMRI_values = list()


# LATE FUSION
for d in tqdm(fusion_loader):
    acc_id = d['ACCESSION_ID']
    for val in acc_id:
        acc_ids.append(val)
    
    gt_segm = d['GT-SEGM']
    pre_segm = d['PRE-SEGM']
    art_segm = d['ART-SEGM']
    pv_segm = d['PV-SEGM']
    del_segm = d['DEL-SEGM']
    
    #print(art_segm.shape)
    #print(pre_segm.shape)
    
    # Do all numpy
    
    # 6 x 2-mpMRIs
    segm_2mpMRI_pre_art = torch.mean(torch.cat([pre_segm, art_segm]), axis = 0, keepdims=True)
    # Threshold at 0.5 (like for MCDropout)
    segm_2mpMRI_pre_art = torch.where(segm_2mpMRI_pre_art >= 0.5, 1, 0)
 
    segm_2mpMRI_pre_pv = torch.mean(torch.cat([pre_segm, pv_segm]), axis = 0, keepdims=True)
    # Threshold at 0.5 (like for MCDropout)
    segm_2mpMRI_pre_pv = torch.where(segm_2mpMRI_pre_pv >= 0.5, 1, 0)

    segm_2mpMRI_pre_del = torch.mean(torch.cat([pre_segm, del_segm]), axis = 0, keepdims=True)
    # Threshold at 0.5 (like for MCDropout)
    segm_2mpMRI_pre_del = torch.where(segm_2mpMRI_pre_del >= 0.5, 1, 0)
    
    segm_2mpMRI_art_pv = torch.mean(torch.cat([art_segm, pv_segm]), axis = 0, keepdims=True)
    # Threshold at 0.5 (like for MCDropout)
    segm_2mpMRI_art_pv = torch.where(segm_2mpMRI_art_pv >= 0.5, 1, 0)
    
    segm_2mpMRI_art_del = torch.mean(torch.cat([art_segm, del_segm]), axis = 0, keepdims=True)
    # Threshold at 0.5 (like for MCDropout)
    segm_2mpMRI_art_del = torch.where(segm_2mpMRI_art_del >= 0.5, 1, 0)
    
    segm_2mpMRI_pv_del = torch.mean(torch.cat([pv_segm, del_segm]), axis = 0, keepdims=True)
    # Threshold at 0.5 (like for MCDropout)
    segm_2mpMRI_pv_del = torch.where(segm_2mpMRI_pv_del >= 0.5, 1, 0)


    
    # 4 x 3-mpMRIs
    
    segm_3mpMRI_pre_art_pv = torch.mean(torch.cat([pre_segm, art_segm, pv_segm]), axis = 0, keepdims=True)
    # Threshold at 0.5 (like for MCDropout)
    segm_3mpMRI_pre_art_pv = torch.where(segm_3mpMRI_pre_art_pv >= 0.5, 1, 0)
    
    segm_3mpMRI_pre_art_del = torch.mean(torch.cat([pre_segm, art_segm, del_segm]), axis = 0, keepdims=True)
    # Threshold at 0.5 (like for MCDropout)
    segm_3mpMRI_pre_art_del = torch.where(segm_3mpMRI_pre_art_del >= 0.5, 1, 0)
    
    segm_3mpMRI_pre_pv_del = torch.mean(torch.cat([pre_segm, pv_segm, del_segm]), axis = 0, keepdims=True)
    # Threshold at 0.5 (like for MCDropout)
    segm_3mpMRI_pre_pv_del = torch.where(segm_3mpMRI_pre_pv_del >= 0.5, 1, 0)
    
    segm_3mpMRI_art_pv_del = torch.mean(torch.cat([art_segm, pv_segm, del_segm]), axis = 0, keepdims=True)
    # Threshold at 0.5 (like for MCDropout)
    segm_3mpMRI_art_pv_del = torch.where(segm_3mpMRI_art_pv_del >= 0.5, 1, 0)


    
    
    # 1 x 4-mpMRI
    
    segm_4mpMRI = torch.mean(torch.cat([pre_segm,art_segm, pv_segm, del_segm]), axis = 0, keepdims=True)
    # Threshold at 0.5 (like for MCDropout)
    segm_4mpMRI = torch.where(segm_4mpMRI >= 0.5, 1, 0)

    
    
    
    
    #2 mpMRI
    # Compute metrics comparing fusion result to ground-truth
    dice_values, hd_values, mad_values = compute_evaluation_metrics(segm_2mpMRI_pre_art, gt_segm)
   
    for val in dice_values:
        dice_segm_2mpMRI_pre_art_values.append(val)
    for val in hd_values:
        hd_segm_2mpMRI_pre_art_values.append(val)
    for val in mad_values:
        mad_segm_2mpMRI_pre_art_values.append(val)
        

    dice_values, hd_values, mad_values = compute_evaluation_metrics(segm_2mpMRI_pre_pv, gt_segm)
   
    for val in dice_values:
        dice_segm_2mpMRI_pre_pv_values.append(val)
    for val in hd_values:
        hd_segm_2mpMRI_pre_pv_values.append(val)
    for val in mad_values:
        mad_segm_2mpMRI_pre_pv_values.append(val)
    
    
    dice_values, hd_values, mad_values = compute_evaluation_metrics(segm_2mpMRI_pre_del, gt_segm)
   
    for val in dice_values:
        dice_segm_2mpMRI_pre_del_values.append(val)
    for val in hd_values:
        hd_segm_2mpMRI_pre_del_values.append(val)
    for val in mad_values:
        mad_segm_2mpMRI_pre_del_values.append(val)
        
    
    dice_values, hd_values, mad_values = compute_evaluation_metrics(segm_2mpMRI_art_pv, gt_segm)
   
    for val in dice_values:
        dice_segm_2mpMRI_art_pv_values.append(val)
    for val in hd_values:
        hd_segm_2mpMRI_art_pv_values.append(val)
    for val in mad_values:
        mad_segm_2mpMRI_art_pv_values.append(val)

    dice_values, hd_values, mad_values = compute_evaluation_metrics(segm_2mpMRI_art_del, gt_segm)
   
    for val in dice_values:
        dice_segm_2mpMRI_art_del_values.append(val)
    for val in hd_values:
        hd_segm_2mpMRI_art_del_values.append(val)
    for val in mad_values:
        mad_segm_2mpMRI_art_del_values.append(val)
        
    dice_values, hd_values, mad_values = compute_evaluation_metrics(segm_2mpMRI_pv_del, gt_segm)
   
    for val in dice_values:
        dice_segm_2mpMRI_pv_del_values.append(val)
    for val in hd_values:
        hd_segm_2mpMRI_pv_del_values.append(val)
    for val in mad_values:
        mad_segm_2mpMRI_pv_del_values.append(val)
        
    
    
    #3-mpMRI

     
    dice_values, hd_values, mad_values = compute_evaluation_metrics(segm_3mpMRI_pre_art_pv, gt_segm)
   
    for val in dice_values:
        dice_segm_3mpMRI_pre_art_pv_values.append(val)
    for val in hd_values:
        hd_segm_3mpMRI_pre_art_pv_values.append(val)
    for val in mad_values:
        mad_segm_3mpMRI_pre_art_pv_values.append(val)
        
    dice_values, hd_values, mad_values = compute_evaluation_metrics(segm_3mpMRI_pre_art_del, gt_segm)
   
    for val in dice_values:
        dice_segm_3mpMRI_pre_art_del_values.append(val)
    for val in hd_values:
        hd_segm_3mpMRI_pre_art_del_values.append(val)
    for val in mad_values:
        mad_segm_3mpMRI_pre_art_del_values.append(val)
        
    dice_values, hd_values, mad_values = compute_evaluation_metrics(segm_3mpMRI_pre_pv_del, gt_segm)
   
    for val in dice_values:
        dice_segm_3mpMRI_pre_pv_del_values.append(val)
    for val in hd_values:
        hd_segm_3mpMRI_pre_pv_del_values.append(val)
    for val in mad_values:
        mad_segm_3mpMRI_pre_pv_del_values.append(val)

    dice_values, hd_values, mad_values = compute_evaluation_metrics(segm_3mpMRI_art_pv_del, gt_segm)
   
    for val in dice_values:
        dice_segm_3mpMRI_art_pv_del_values.append(val)
    for val in hd_values:
        hd_segm_3mpMRI_art_pv_del_values.append(val)
    for val in mad_values:
        mad_segm_3mpMRI_art_pv_del_values.append(val)
        
    #4-mpMRI
        
    dice_values, hd_values, mad_values = compute_evaluation_metrics(segm_4mpMRI, gt_segm)
   
    for val in dice_values:
        dice_segm_4mpMRI_values.append(val)
    for val in hd_values:
        hd_segm_4mpMRI_values.append(val)
    for val in mad_values:
        mad_segm_4mpMRI_values.append(val)


 

    


  0%|          | 0/88 [00:00<?, ?it/s]

In [19]:
df_2mp_pre_art_results_late_fusion = pd.DataFrame({ 'ACCESSION_ID' : acc_ids,
                                          'POST_DICE' : dice_segm_2mpMRI_pre_art_values, 
                                          'POST_HD95' : hd_segm_2mpMRI_pre_art_values, 
                                          'POST_MAD' : mad_segm_2mpMRI_pre_art_values,
                                          'EXPERIMENT_ID' : 'LATE-FUSION-2mpMRI_PRE-ART'})
df_2mp_pre_art_results_late_fusion

,ACCESSION_ID,POST_DICE,POST_HD95,POST_MAD,EXPERIMENT_ID
0,E100998852,0.958308,2.000000,0.683238,LATE-FUSION-2mpMRI_PRE-ART
1,12370686,0.972035,1.732051,0.506965,LATE-FUSION-2mpMRI_PRE-ART
2,E104184251,0.968940,1.732051,0.506262,LATE-FUSION-2mpMRI_PRE-ART
3,E100353838,0.928511,4.898979,1.371619,LATE-FUSION-2mpMRI_PRE-ART
4,11655071,0.956169,3.000000,0.680382,LATE-FUSION-2mpMRI_PRE-ART
...,...,...,...,...,...
83,10843891,0.946354,3.464102,0.957132,LATE-FUSION-2mpMRI_PRE-ART
84,E101762106,0.962943,2.236068,0.719885,LATE-FUSION-2mpMRI_PRE-ART
85,12543786,0.956476,2.236068,0.732016,LATE-FUSION-2mpMRI_PRE-ART
86,E107077373,0.941316,3.741657,1.129438,LATE-FUSION-2mpMRI_PRE-ART


## 2 mpMRI

In [20]:
df_2mp_pre_art_results_late_fusion = pd.DataFrame({ 'ACCESSION_ID' : acc_ids,
                                          'POST_DICE' : dice_segm_2mpMRI_pre_art_values, 
                                          'POST_HD95' : hd_segm_2mpMRI_pre_art_values, 
                                          'POST_MAD' : mad_segm_2mpMRI_pre_art_values,
                                          'EXPERIMENT_ID' : 'LATE-FUSION-2mpMRI_PRE-ART'})

df_2mp_pre_art_results_late_fusion.to_csv('/home/mgross/Developement/liver-center/studies/Contrast-Nets/Results/2mpMRI-late-fusion-PRE-ART.csv')
df_2mp_pre_art_results_late_fusion

,ACCESSION_ID,POST_DICE,POST_HD95,POST_MAD,EXPERIMENT_ID
0,E100998852,0.958308,2.000000,0.683238,LATE-FUSION-2mpMRI_PRE-ART
1,12370686,0.972035,1.732051,0.506965,LATE-FUSION-2mpMRI_PRE-ART
2,E104184251,0.968940,1.732051,0.506262,LATE-FUSION-2mpMRI_PRE-ART
3,E100353838,0.928511,4.898979,1.371619,LATE-FUSION-2mpMRI_PRE-ART
4,11655071,0.956169,3.000000,0.680382,LATE-FUSION-2mpMRI_PRE-ART
...,...,...,...,...,...
83,10843891,0.946354,3.464102,0.957132,LATE-FUSION-2mpMRI_PRE-ART
84,E101762106,0.962943,2.236068,0.719885,LATE-FUSION-2mpMRI_PRE-ART
85,12543786,0.956476,2.236068,0.732016,LATE-FUSION-2mpMRI_PRE-ART
86,E107077373,0.941316,3.741657,1.129438,LATE-FUSION-2mpMRI_PRE-ART


In [21]:
df_2mp_pre_pv_results_late_fusion = pd.DataFrame({ 'ACCESSION_ID' : acc_ids,
                                          'POST_DICE' : dice_segm_2mpMRI_pre_pv_values, 
                                          'POST_HD95' : hd_segm_2mpMRI_pre_pv_values, 
                                          'POST_MAD' : mad_segm_2mpMRI_pre_pv_values,
                                          'EXPERIMENT_ID' : 'LATE-FUSION-2mpMRI_PRE-PV'})

df_2mp_pre_pv_results_late_fusion.to_csv('/home/mgross/Developement/liver-center/studies/Contrast-Nets/Results/2mpMRI-late-fusion-PRE-PV.csv')
df_2mp_pre_pv_results_late_fusion

,ACCESSION_ID,POST_DICE,POST_HD95,POST_MAD,EXPERIMENT_ID
0,E100998852,0.964685,2.000000,0.581872,LATE-FUSION-2mpMRI_PRE-PV
1,12370686,0.977107,1.414214,0.422258,LATE-FUSION-2mpMRI_PRE-PV
2,E104184251,0.973789,1.414214,0.420423,LATE-FUSION-2mpMRI_PRE-PV
3,E100353838,0.932059,4.867695,1.299613,LATE-FUSION-2mpMRI_PRE-PV
4,11655071,0.960314,2.828427,0.627257,LATE-FUSION-2mpMRI_PRE-PV
...,...,...,...,...,...
83,10843891,0.953721,2.828427,0.845023,LATE-FUSION-2mpMRI_PRE-PV
84,E101762106,0.974488,1.414214,0.489019,LATE-FUSION-2mpMRI_PRE-PV
85,12543786,0.964228,2.000000,0.609818,LATE-FUSION-2mpMRI_PRE-PV
86,E107077373,0.964821,2.000000,0.544639,LATE-FUSION-2mpMRI_PRE-PV


In [22]:
df_2mp_pre_del_results_late_fusion = pd.DataFrame({ 'ACCESSION_ID' : acc_ids,
                                          'POST_DICE' : dice_segm_2mpMRI_pre_del_values, 
                                          'POST_HD95' : hd_segm_2mpMRI_pre_del_values, 
                                          'POST_MAD' : mad_segm_2mpMRI_pre_del_values,
                                          'EXPERIMENT_ID' : 'LATE-FUSION-2mpMRI_PRE-DEL'})

df_2mp_pre_del_results_late_fusion.to_csv('/home/mgross/Developement/liver-center/studies/Contrast-Nets/Results/2mpMRI-late-fusion-PRE-DEL.csv')
df_2mp_pre_del_results_late_fusion

,ACCESSION_ID,POST_DICE,POST_HD95,POST_MAD,EXPERIMENT_ID
0,E100998852,0.960406,2.000000,0.651934,LATE-FUSION-2mpMRI_PRE-DEL
1,12370686,0.975294,1.414214,0.458918,LATE-FUSION-2mpMRI_PRE-DEL
2,E104184251,0.972019,1.414214,0.453169,LATE-FUSION-2mpMRI_PRE-DEL
3,E100353838,0.929182,4.898979,1.353889,LATE-FUSION-2mpMRI_PRE-DEL
4,11655071,0.959666,3.000000,0.633208,LATE-FUSION-2mpMRI_PRE-DEL
...,...,...,...,...,...
83,10843891,0.946674,3.162278,0.963876,LATE-FUSION-2mpMRI_PRE-DEL
84,E101762106,0.956118,3.000000,0.831094,LATE-FUSION-2mpMRI_PRE-DEL
85,12543786,0.956064,2.828427,0.756250,LATE-FUSION-2mpMRI_PRE-DEL
86,E107077373,0.956736,2.828427,0.775044,LATE-FUSION-2mpMRI_PRE-DEL


In [23]:
df_2mp_art_pv_results_late_fusion = pd.DataFrame({ 'ACCESSION_ID' : acc_ids,
                                          'POST_DICE' : dice_segm_2mpMRI_art_pv_values, 
                                          'POST_HD95' : hd_segm_2mpMRI_art_pv_values, 
                                          'POST_MAD' : mad_segm_2mpMRI_art_pv_values,
                                          'EXPERIMENT_ID' : 'LATE-FUSION-2mpMRI_ART-PV'})

df_2mp_art_pv_results_late_fusion.to_csv('/home/mgross/Developement/liver-center/studies/Contrast-Nets/Results/2mpMRI-late-fusion-ART-PV.csv')
df_2mp_art_pv_results_late_fusion

,ACCESSION_ID,POST_DICE,POST_HD95,POST_MAD,EXPERIMENT_ID
0,E100998852,0.965591,1.745448,0.545953,LATE-FUSION-2mpMRI_ART-PV
1,12370686,0.975481,1.414214,0.440658,LATE-FUSION-2mpMRI_ART-PV
2,E104184251,0.972762,1.414214,0.443961,LATE-FUSION-2mpMRI_ART-PV
3,E100353838,0.975653,2.000000,0.486750,LATE-FUSION-2mpMRI_ART-PV
4,11655071,0.962524,2.236068,0.588717,LATE-FUSION-2mpMRI_ART-PV
...,...,...,...,...,...
83,10843891,0.960804,2.828427,0.685061,LATE-FUSION-2mpMRI_ART-PV
84,E101762106,0.965314,2.236068,0.670218,LATE-FUSION-2mpMRI_ART-PV
85,12543786,0.962536,2.000000,0.613013,LATE-FUSION-2mpMRI_ART-PV
86,E107077373,0.944748,3.605551,1.079691,LATE-FUSION-2mpMRI_ART-PV


In [24]:
df_2mp_art_del_results_late_fusion = pd.DataFrame({ 'ACCESSION_ID' : acc_ids,
                                          'POST_DICE' : dice_segm_2mpMRI_art_del_values, 
                                          'POST_HD95' : hd_segm_2mpMRI_art_del_values, 
                                          'POST_MAD' : mad_segm_2mpMRI_art_del_values,
                                          'EXPERIMENT_ID' : 'LATE-FUSION-2mpMRI_ART-DEL'})

df_2mp_art_del_results_late_fusion.to_csv('/home/mgross/Developement/liver-center/studies/Contrast-Nets/Results/2mpMRI-late-fusion-ART-DEL.csv')
df_2mp_art_del_results_late_fusion

,ACCESSION_ID,POST_DICE,POST_HD95,POST_MAD,EXPERIMENT_ID
0,E100998852,0.961867,2.000000,0.605733,LATE-FUSION-2mpMRI_ART-DEL
1,12370686,0.974182,1.414214,0.462564,LATE-FUSION-2mpMRI_ART-DEL
2,E104184251,0.971148,1.414214,0.472148,LATE-FUSION-2mpMRI_ART-DEL
3,E100353838,0.972767,2.000000,0.538523,LATE-FUSION-2mpMRI_ART-DEL
4,11655071,0.961500,2.828427,0.602350,LATE-FUSION-2mpMRI_ART-DEL
...,...,...,...,...,...
83,10843891,0.954886,3.000000,0.788558,LATE-FUSION-2mpMRI_ART-DEL
84,E101762106,0.952098,3.000000,0.931129,LATE-FUSION-2mpMRI_ART-DEL
85,12543786,0.954705,2.449490,0.757124,LATE-FUSION-2mpMRI_ART-DEL
86,E107077373,0.937457,5.744563,1.278803,LATE-FUSION-2mpMRI_ART-DEL


In [25]:
df_2mp_pv_del_results_late_fusion = pd.DataFrame({ 'ACCESSION_ID' : acc_ids,
                                          'POST_DICE' : dice_segm_2mpMRI_pv_del_values, 
                                          'POST_HD95' : hd_segm_2mpMRI_pv_del_values, 
                                          'POST_MAD' : mad_segm_2mpMRI_pv_del_values,
                                          'EXPERIMENT_ID' : 'LATE-FUSION-2mpMRI_PV-DEL'})

df_2mp_pv_del_results_late_fusion.to_csv('/home/mgross/Developement/liver-center/studies/Contrast-Nets/Results/2mpMRI-late-fusion-PV-DEL.csv')
df_2mp_pv_del_results_late_fusion

,ACCESSION_ID,POST_DICE,POST_HD95,POST_MAD,EXPERIMENT_ID
0,E100998852,0.969145,1.414214,0.486259,LATE-FUSION-2mpMRI_PV-DEL
1,12370686,0.979159,1.414214,0.381826,LATE-FUSION-2mpMRI_PV-DEL
2,E104184251,0.976347,1.414214,0.380027,LATE-FUSION-2mpMRI_PV-DEL
3,E100353838,0.977239,1.414214,0.449134,LATE-FUSION-2mpMRI_PV-DEL
4,11655071,0.967754,2.000000,0.518229,LATE-FUSION-2mpMRI_PV-DEL
...,...,...,...,...,...
83,10843891,0.964028,2.000000,0.654215,LATE-FUSION-2mpMRI_PV-DEL
84,E101762106,0.964185,2.828427,0.689955,LATE-FUSION-2mpMRI_PV-DEL
85,12543786,0.964548,2.236068,0.592096,LATE-FUSION-2mpMRI_PV-DEL
86,E107077373,0.960592,2.236068,0.709339,LATE-FUSION-2mpMRI_PV-DEL


# 3-mpMRI

In [26]:
df_3mp_pre_art_pv_results_late_fusion = pd.DataFrame({ 'ACCESSION_ID' : acc_ids,
                                          'POST_DICE' : dice_segm_3mpMRI_pre_art_pv_values, 
                                          'POST_HD95' : hd_segm_3mpMRI_pre_art_pv_values, 
                                          'POST_MAD' : mad_segm_3mpMRI_pre_art_pv_values,
                                          'EXPERIMENT_ID' : 'LATE-FUSION-3mpMRI_PRE-ART-PV'})

df_3mp_pre_art_pv_results_late_fusion.to_csv('/home/mgross/Developement/liver-center/studies/Contrast-Nets/Results/3mpMRI-late-fusion-PRE-ART-PV.csv')
df_3mp_pre_art_pv_results_late_fusion

,ACCESSION_ID,POST_DICE,POST_HD95,POST_MAD,EXPERIMENT_ID
0,E100998852,0.969661,1.414214,0.476934,LATE-FUSION-3mpMRI_PRE-ART-PV
1,12370686,0.980917,1.000000,0.347327,LATE-FUSION-3mpMRI_PRE-ART-PV
2,E104184251,0.975739,1.414214,0.382721,LATE-FUSION-3mpMRI_PRE-ART-PV
3,E100353838,0.979005,1.414214,0.409269,LATE-FUSION-3mpMRI_PRE-ART-PV
4,11655071,0.962747,2.828427,0.565445,LATE-FUSION-3mpMRI_PRE-ART-PV
...,...,...,...,...,...
83,10843891,0.966492,2.000000,0.589957,LATE-FUSION-3mpMRI_PRE-ART-PV
84,E101762106,0.974752,1.414214,0.473281,LATE-FUSION-3mpMRI_PRE-ART-PV
85,12543786,0.973660,1.414214,0.429249,LATE-FUSION-3mpMRI_PRE-ART-PV
86,E107077373,0.963927,2.000000,0.539647,LATE-FUSION-3mpMRI_PRE-ART-PV


In [27]:
df_3mp_pre_art_del_results_late_fusion = pd.DataFrame({ 'ACCESSION_ID' : acc_ids,
                                          'POST_DICE' : dice_segm_3mpMRI_pre_art_del_values, 
                                          'POST_HD95' : hd_segm_3mpMRI_pre_art_del_values, 
                                          'POST_MAD' : mad_segm_3mpMRI_pre_art_del_values,
                                          'EXPERIMENT_ID' : 'LATE-FUSION-3mpMRI_PRE-ART-DEL'})

df_3mp_pre_art_del_results_late_fusion.to_csv('/home/mgross/Developement/liver-center/studies/Contrast-Nets/Results/3mpMRI-late-fusion-PRE-ART-DEL.csv')
df_3mp_pre_art_del_results_late_fusion

,ACCESSION_ID,POST_DICE,POST_HD95,POST_MAD,EXPERIMENT_ID
0,E100998852,0.967232,1.732051,0.515985,LATE-FUSION-3mpMRI_PRE-ART-DEL
1,12370686,0.979681,1.414214,0.374900,LATE-FUSION-3mpMRI_PRE-ART-DEL
2,E104184251,0.973767,1.414214,0.416531,LATE-FUSION-3mpMRI_PRE-ART-DEL
3,E100353838,0.975478,1.732051,0.474145,LATE-FUSION-3mpMRI_PRE-ART-DEL
4,11655071,0.963421,2.828427,0.558364,LATE-FUSION-3mpMRI_PRE-ART-DEL
...,...,...,...,...,...
83,10843891,0.961096,2.449490,0.681035,LATE-FUSION-3mpMRI_PRE-ART-DEL
84,E101762106,0.968615,1.732051,0.585662,LATE-FUSION-3mpMRI_PRE-ART-DEL
85,12543786,0.970397,1.414214,0.484772,LATE-FUSION-3mpMRI_PRE-ART-DEL
86,E107077373,0.960901,2.000000,0.595743,LATE-FUSION-3mpMRI_PRE-ART-DEL


In [28]:
df_3mp_pre_pv_del_results_late_fusion = pd.DataFrame({ 'ACCESSION_ID' : acc_ids,
                                          'POST_DICE' : dice_segm_3mpMRI_pre_pv_del_values, 
                                          'POST_HD95' : hd_segm_3mpMRI_pre_pv_del_values, 
                                          'POST_MAD' : mad_segm_3mpMRI_pre_pv_del_values,
                                          'EXPERIMENT_ID' : 'LATE-FUSION-3mpMRI_PRE-PV-DEL'})

df_3mp_pre_pv_del_results_late_fusion.to_csv('/home/mgross/Developement/liver-center/studies/Contrast-Nets/Results/3mpMRI-late-fusion-PRE-PV-DEL.csv')
df_3mp_pre_pv_del_results_late_fusion

,ACCESSION_ID,POST_DICE,POST_HD95,POST_MAD,EXPERIMENT_ID
0,E100998852,0.972357,1.414214,0.429731,LATE-FUSION-3mpMRI_PRE-PV-DEL
1,12370686,0.981438,1.000000,0.341421,LATE-FUSION-3mpMRI_PRE-PV-DEL
2,E104184251,0.976287,1.414214,0.375224,LATE-FUSION-3mpMRI_PRE-PV-DEL
3,E100353838,0.977796,1.414214,0.425256,LATE-FUSION-3mpMRI_PRE-PV-DEL
4,11655071,0.966355,2.236068,0.517804,LATE-FUSION-3mpMRI_PRE-PV-DEL
...,...,...,...,...,...
83,10843891,0.968128,1.732051,0.580096,LATE-FUSION-3mpMRI_PRE-PV-DEL
84,E101762106,0.970859,1.414214,0.540188,LATE-FUSION-3mpMRI_PRE-PV-DEL
85,12543786,0.971997,1.414214,0.451034,LATE-FUSION-3mpMRI_PRE-PV-DEL
86,E107077373,0.967837,1.732051,0.486738,LATE-FUSION-3mpMRI_PRE-PV-DEL


In [29]:
df_3mp_art_pv_del_results_late_fusion = pd.DataFrame({ 'ACCESSION_ID' : acc_ids,
                                          'POST_DICE' : dice_segm_3mpMRI_art_pv_del_values, 
                                          'POST_HD95' : hd_segm_3mpMRI_art_pv_del_values, 
                                          'POST_MAD' : mad_segm_3mpMRI_art_pv_del_values,
                                          'EXPERIMENT_ID' : 'LATE-FUSION-3mpMRI_ART-PV-DEL'})

df_3mp_art_pv_del_results_late_fusion.to_csv('/home/mgross/Developement/liver-center/studies/Contrast-Nets/Results/3mpMRI-late-fusion-ART-PV-DEL.csv')
df_3mp_art_pv_del_results_late_fusion

,ACCESSION_ID,POST_DICE,POST_HD95,POST_MAD,EXPERIMENT_ID
0,E100998852,0.971296,1.414214,0.444859,LATE-FUSION-3mpMRI_ART-PV-DEL
1,12370686,0.980386,1.000000,0.357664,LATE-FUSION-3mpMRI_ART-PV-DEL
2,E104184251,0.976457,1.414214,0.371819,LATE-FUSION-3mpMRI_ART-PV-DEL
3,E100353838,0.979729,1.414214,0.400467,LATE-FUSION-3mpMRI_ART-PV-DEL
4,11655071,0.967050,2.236068,0.509241,LATE-FUSION-3mpMRI_ART-PV-DEL
...,...,...,...,...,...
83,10843891,0.968624,2.000000,0.565629,LATE-FUSION-3mpMRI_ART-PV-DEL
84,E101762106,0.976620,1.414214,0.436994,LATE-FUSION-3mpMRI_ART-PV-DEL
85,12543786,0.973009,1.414214,0.431785,LATE-FUSION-3mpMRI_ART-PV-DEL
86,E107077373,0.966039,1.732051,0.514568,LATE-FUSION-3mpMRI_ART-PV-DEL


# 4-mpMRI

In [33]:
acc_ids = list()


#dsc
dice_segm_4mpMRI_values = list()

#hd
hd_segm_4mpMRI_values = list()

#mad
mad_segm_4mpMRI_values = list()


# LATE FUSION
for d in tqdm(fusion_loader):
    acc_id = d['ACCESSION_ID']
    for val in acc_id:
        acc_ids.append(val)
    
    gt_segm = d['GT-SEGM']
    pre_segm = d['PRE-SEGM']
    art_segm = d['ART-SEGM']
    pv_segm = d['PV-SEGM']
    del_segm = d['DEL-SEGM']

    
    
    # 1 x 4-mpMRI
    
    segm_4mpMRI = torch.mean(torch.cat([pre_segm,art_segm, pv_segm, del_segm]), axis = 0, keepdims=True)
    # Threshold at 0.5 (like for MCDropout)
    segm_4mpMRI = torch.where(segm_4mpMRI >= 0.5, 1, 0)

 
    #4-mpMRI
        
    dice_values, hd_values, mad_values = compute_evaluation_metrics(segm_4mpMRI, gt_segm)
   
    for val in dice_values:
        dice_segm_4mpMRI_values.append(val)
    for val in hd_values:
        hd_segm_4mpMRI_values.append(val)
    for val in mad_values:
        mad_segm_4mpMRI_values.append(val)
        


 

    


  0%|          | 0/88 [00:00<?, ?it/s]

In [34]:
df_4mp_results_late_fusion = pd.DataFrame({'ACCESSION_ID' : acc_ids,
                                           'POST_DICE' : dice_segm_4mpMRI_values, 
                                          'POST_HD95' : hd_segm_4mpMRI_values, 
                                          'POST_MAD' : mad_segm_4mpMRI_values,
                                          'EXPERIMENT_ID' : 'LATE-FUSION-4mpMRI'})
df_4mp_results_late_fusion

,ACCESSION_ID,POST_DICE,POST_HD95,POST_MAD,EXPERIMENT_ID
0,E100998852,0.968642,1.414214,0.497571,LATE-FUSION-4mpMRI
1,12370686,0.979460,1.414214,0.376608,LATE-FUSION-4mpMRI
2,E104184251,0.975455,1.414214,0.391264,LATE-FUSION-4mpMRI
3,E100353838,0.977201,1.732051,0.449714,LATE-FUSION-4mpMRI
4,11655071,0.964231,2.449490,0.553315,LATE-FUSION-4mpMRI
...,...,...,...,...,...
83,10843891,0.962726,2.236068,0.656851,LATE-FUSION-4mpMRI
84,E101762106,0.974563,1.414214,0.486450,LATE-FUSION-4mpMRI
85,12543786,0.969770,1.732051,0.499177,LATE-FUSION-4mpMRI
86,E107077373,0.963082,2.000000,0.573481,LATE-FUSION-4mpMRI


In [35]:
df_4mp_results_late_fusion.to_csv('/home/mgross/Developement/liver-center/studies/Contrast-Nets/Results/4mpMRI-late-fusion.csv')
